In [31]:
import altair as alt, pandas as pd, json

In [2]:
# ECO API has 12-month inflation, we want 1-month updates
# url = 'https://api.economicsobservatory.com/usa/infl?vega'


# Use FRED API to get 1-month inflation
# Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL)
series = 'CPIAUCSL'

In [7]:
def get_fred_data(series):
    import requests
    import urllib.parse
    # Fetch data from the FRED API
    api_url = 'https://api.stlouisfed.org/fred/series/observations'
    params = {
        'series_id': series,
        'api_key': '838a40e4f5a37b6b4d8c9cfc4b1abaff',
        'file_type': 'json',
    }
    encoded_params = urllib.parse.urlencode(params)
    api_query_url = f"https://api.allorigins.win/raw?url={urllib.parse.quote(api_url + '?' + encoded_params)}"

    # Send a GET request to the API
    response = requests.get(api_query_url)

    # Extract data from the 'months' key
    months_data = response.json()['observations']

    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame.from_dict(months_data)

    # drop columns `realtime_start` and `realtime_end` if they exist
    
    df.drop(columns=['realtime_start', 'realtime_end'], inplace=True, errors='ignore')

    df['value'] = df['value'].astype(float)

    return df

In [42]:
df = get_fred_data(series)

In [43]:
# `df` contains CPI data where index 1982-1984=100
# let's calculate the monthly year-on-year inflation rate

df['value'] = df['value'].pct_change(periods=12).dropna()

In [44]:
# Create Altair chart

chart = alt.Chart(df).mark_line().encode(
    x='date:T',
    y='value:Q',
    tooltip=['date:T', 'value:Q'],
).properties(
    title='US inflation rate (year-on-year)'
)

In [45]:
chart.display()

alt.Chart(...)

In [46]:
dark = True

background = '#122b39' if dark else '#b4c8d8'
detail = '#b4c8d8' if dark else '#122b39'
line_colour = '#e6224b'

In [47]:
# create chart using Altair

chart = alt.Chart(df[df['date'] >= '2015-01-01']).mark_line(color=line_colour).encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            labelColor=detail,
            tickColor=detail,
            domainColor=detail,
            gridColor=detail,
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title='12-month percent change | Source: BLS via FRED',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor=detail,
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7,
            format='%'
        ),
        scale=alt.Scale(zero=False, padding=25)
    )
).properties(
    width = 350,
    height = 280,
    title = 'US Inflation'
)

# Add a point on at the last data point
# First filter the data to get the last data point
# Then add a point on the chart
last_point = df[df['date'] == df['date'].max()]

point = alt.Chart(last_point).mark_point(color=line_colour, size=60, opacity=0.9, filled=True).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q')
)

# Now let's add text to the point
# add a column to `last_point` dataframe with the text we want to display
last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1%}')

# Add a column with the month name `August`
last_point['month'] = 'September'

text_value = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=0,
    color=detail,
    fontSize=14,
).encode(
    text='text'
)

text_month = point.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    dy=-16,
    color=detail,
    fontSize=14,
).encode(
    text='month'
)

# add a line at y=0
line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color=detail, size=1.5, opacity=0.7).encode(y='y')

chart = line + chart + point + text_month + text_value


chart = chart.configure(
    font='Circular Std',
    background=background
).configure_title(
    anchor='start',
    dx=24,
    fontSize=12,
    subtitleFontSize=12,
    color=detail,
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_31751/394998334.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_point['text'] = last_point['value'].apply(lambda x: f'{x:.1%}')
/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_31751/394998334.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_point['month'] = 'September'


alt.LayerChart(...)

In [48]:
df

,date,value
0,1947-01-01,NaN
1,1947-02-01,NaN
2,1947-03-01,NaN
3,1947-04-01,NaN
4,1947-05-01,NaN
...,...,...
916,2023-05-01,0.041288
917,2023-06-01,0.030920
918,2023-07-01,0.032991
919,2023-08-01,0.037075


In [49]:
# Convert chart to dictionary
chart_dict = chart.to_dict()

# Modify width and height directly (Altair will set continuousWidth and height by default)
chart_dict['width'] = 350
chart_dict['height'] = 280

# Convert the dictionary back to JSON
vega_spec = json.dumps(chart_dict, indent=2)

# Write the JSON to a file
with open('../../charts/202310/20231012_US_CPI_Rate_12month.json', 'w') as f:
    f.write(vega_spec)